<a href="https://colab.research.google.com/github/jeguzmanmd/BIOE464/blob/master/Yolo_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Modules
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
## provide the path for testing config file and trained model from google colab
net = cv2.dnn.readNetFromDarknet("yolov3_custom.cfg",r"yolov3_custom_10000.weights")


In [ ]:
# Classes
classes = ['Right','Transverse','Left','Sigmoid','Rectum']

In [ ]:
## Capture of the image
my_img=cv2.imread('80.jpg')
my_img=cv2.resize(my_img,(1280,720))

In [ ]:
plt.imshow(my_img)

In [ ]:
# Referecnce for boxes coordinates
ht, wt, _ = my_img.shape

In [ ]:
# Set up of the image for analysis
blob = cv2.dnn.blobFromImage(my_img, 1/255,(416,416),(0,0,0),swapRB = True,crop= False)

In [ ]:
blob.shape

In [ ]:
# Location of detected objects and Application of confidences 
net.setInput(blob)

output_layers_name = net.getUnconnectedOutLayersNames()

layerOutputs = net.forward(output_layers_name)

boxes =[]
confidences = []
class_ids = []

for output in layerOutputs:
    for detection in output:
        score = detection[5:]
        class_id = np.argmax(score)
        confidence = score[class_id]
        if confidence > 0.2:
            center_x = int(detection[0] * wt)
            center_y = int(detection[1] * ht)
            w = int(detection[2] * wt)
            h = int(detection[3]* ht)
            x = int(center_x - w/2)
            y = int(center_y - h/2)
            boxes.append([x,y,w,h])
            confidences.append((float(confidence)))
            class_ids.append(class_id)
            

In [ ]:
indexes = cv2.dnn.NMSBoxes(boxes,confidences,.2,.2)
font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0,255,size =(len(boxes),3))
if  len(indexes)>0:
    for i in indexes.flatten():
        x,y,w,h = boxes[i]
        label = str(classes[class_ids[i]])
        confidence = str(round(confidences[i],2))
        color = colors[i]
        cv2.rectangle(my_img,(x,y),(x+w,y+h),color,2)
        cv2.putText(my_img,label + " " + confidence, (x,y+100),font,2,color,2)
        print(boxes, class_ids[i])

In [ ]:
cv2.imshow('img',my_img)
filename = 'imgbox.jpg'
cv2.imwrite(filename, my_img)
cv2.waitKey(0)
cv2.destroyAllWindows()